In [1]:
import rasterio
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from rasterio.plot import show
import rasterio
import numpy as np
import requests

In [2]:
# # update the wd path to be able to laod the modules
# # os.getcwd()
os.chdir('..')

In [3]:
os.environ['GCS_BUCKET']='gee-exports'
os.environ['GOOGLE_APPLICATION_USER']='developers@citiesindicators.iam.gservaiceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/Users/jt/dev/wri-notebooks/server/citiesindicators-credentials.json'

In [4]:
from city_metrix import (
    built_land_without_tree_cover, 
    mean_tree_cover, 
    built_land_with_low_surface_reflectivity, 
    built_land_with_high_land_surface_temperature
)


Authenticating to GEE with configured credentials file.


# Select a city

In [ ]:
# Get city boundary
cities_api_url = "https://cities-data-api.wri.org/cities"
city_name = "BRA-Salvador"
admin_level = "ADM4union"
city_geom = cities_api_url + "/"+city_name+"/"+admin_level+"/geojson"
city_gdf = gpd.read_file(city_geom)
city_gdf.head()


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."


# Calculate indicators - AOI level

## Compute indicators: One city * One indicator

### HEA-3 Built land with Low Surface reflectivity

In [6]:
%%time

city_gdf["built_land_with_low_surface_reflectivity"] = built_land_with_low_surface_reflectivity(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 5.76 ss
Calculating albedo layer:
[########################################] | 100% Completed | 51.22 s
Extracting ESA world cover layer:
[########################################] | 100% Completed | 7.05 ss
CPU times: user 20.8 s, sys: 3.9 s, total: 24.7 s
Wall time: 1min 13s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551


### LND-2 Land with Tree cover

In [7]:
%%time

city_gdf["mean_tree_cover"] = mean_tree_cover(city_gdf) 
city_gdf

Extracting tree cover layer:
[########################################] | 100% Completed | 9.21 ss
CPU times: user 7.34 s, sys: 1.53 s, total: 8.87 s
Wall time: 12.3 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity,mean_tree_cover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551,28.366773


### HEA-4 Percent of Built land without tree cover

In [8]:
%%time

city_gdf["built_land_without_tree_cover"] = built_land_without_tree_cover(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 6.47 ss
Extracting tree cover layer:
[########################################] | 100% Completed | 8.74 ss
Extracting ESA world cover layer:
[########################################] | 100% Completed | 5.56 ss
CPU times: user 17.8 s, sys: 3.17 s, total: 21 s
Wall time: 27.6 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity,mean_tree_cover,built_land_without_tree_cover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551,28.366773,0.907382


### HEA-2 Built Land With High LST

In [9]:
%%time

city_gdf["built_land_with_high_land_surface_temperature"] = built_land_with_high_land_surface_temperature(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 6.01 ss
Calculating land surface temperature layer:
[########################################] | 100% Completed | 1.82 ss
Extracting ESA world cover layer:
[########################################] | 100% Completed | 5.71 ss
CPU times: user 11 s, sys: 1.72 s, total: 12.7 s
Wall time: 18.5 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity,mean_tree_cover,built_land_without_tree_cover,built_land_with_high_land_surface_temperature
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551,28.366773,0.907382,0.096108


## Compute indicators: One city * Multiple indicators

In [40]:
%%time

indicators =get_indicators(city_gdf, indicators=[Indicator.TREE_COVER,
                                                 Indicator.BUILT_LAND_WITH_TREE_COVER])
indicators_df = pd.merge(indicators[0], indicators[1])
indicators_df

CPU times: total: 3.31 s
Wall time: 8.63 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,LND_2_percentTreeCover,HEA_4_percentBuiltupWithoutTreeCover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.283478,0.907352


# Caculate indicators - Subcity levels

# Get layers

## Admin Boundary

## Esa World Cover

## Albedo

## Trpical Tree cover

## LST